In [ ]:
@file:DependsOn("/data/tools/actin-personalization/actin-personalization.jar")

In [ ]:
import com.hartwig.actin.personalization.datamodel.MetastasesDetectionStatus
import com.hartwig.actin.personalization.datamodel.Episode
import com.hartwig.actin.personalization.datamodel.ReferencePatient
import com.hartwig.actin.personalization.datamodel.Treatment
import com.hartwig.actin.personalization.datamodel.serialization.ReferencePatientJson
import com.hartwig.actin.personalization.ncr.interpretation.ReferencePatientFactory

fun Episode.doesNotIncludeAdjuvantOrNeoadjuvantTreatment(): Boolean {
    return !hasHadPreSurgerySystemicChemotherapy &&
            !hasHadPostSurgerySystemicChemotherapy &&
            !hasHadPreSurgerySystemicTargetedTherapy &&
            !hasHadPostSurgerySystemicTargetedTherapy
}

val patients = ReferencePatientJson.read("/data/patient_like_me/ncr/patientRecords.json")

val referencePop = patients.flatMap(ReferencePatient::tumorEntries).map { (diagnosis, episodes) ->
    diagnosis to episodes.single { it.order == 1 }
}
    .filter { (_, episode) ->
        episode.distantMetastasesDetectionStatus == MetastasesDetectionStatus.AT_START &&
        episode.systemicTreatmentPlan?.treatment?.let { it != Treatment.OTHER } == true &&
        episode.surgeries.isEmpty() &&
        episode.doesNotIncludeAdjuvantOrNeoadjuvantTreatment() &&
        episode.systemicTreatmentPlan?.observedPfsDays != null
    }
    .sortedBy { it.second.systemicTreatmentPlan!!.observedPfsDays!! }

val patientsByTreatment = referencePop.groupBy { (_, episode) ->
    episode.systemicTreatmentPlan!!.treatment.treatmentGroup
}
    .toList()
    .sortedByDescending { it.second.size }

In [ ]:
import com.hartwig.actin.personalization.similarity.population.DiagnosisAndEpisode

data class EventCountAndSurvivalAtTime(val daysSincePlanStart: Int, val numEvents: Int, val survival: Double)
    
tailrec fun eventAndCensorshipHistories(
    populationToProcess: List<DiagnosisAndEpisode>,
    eventHistory: List<EventCountAndSurvivalAtTime> = emptyList(),
    censorshipHistory: List<EventCountAndSurvivalAtTime> = emptyList()
): Pair<List<EventCountAndSurvivalAtTime>, List<EventCountAndSurvivalAtTime>> {
    return if (populationToProcess.isEmpty()) {
        eventHistory to censorshipHistory
    } else {
        val treatmentDetails = populationToProcess.first().second.systemicTreatmentPlan!!
        val previousEvent = eventHistory.lastOrNull() ?: EventCountAndSurvivalAtTime(0, 0, 1.0)
        val (newEventHistory, newCensorshipHistory) = if (treatmentDetails.hadProgressionEvent!!) {
            val newEvent = EventCountAndSurvivalAtTime(
                treatmentDetails.observedPfsDays!!, previousEvent.numEvents + 1, previousEvent.survival * (1 - (1.0 / populationToProcess.size))
            )
            Pair(eventHistory + newEvent, censorshipHistory)
        } else {
            val newCensorship = EventCountAndSurvivalAtTime(treatmentDetails.observedPfsDays!!, previousEvent.numEvents, previousEvent.survival)
            Pair(eventHistory, censorshipHistory + newCensorship)
        }
        eventAndCensorshipHistories(populationToProcess.drop(1), newEventHistory, newCensorshipHistory)
    }
}

val (eventHistory, censorshipHistory) = eventAndCensorshipHistories(referencePop)

In [ ]:
%use dataframe
import com.hartwig.actin.personalization.datamodel.LocationGroup
import com.hartwig.actin.personalization.similarity.population.DiagnosisAndEpisode
import com.hartwig.actin.personalization.similarity.population.PopulationDefinition

val popDefinitions = PopulationDefinition.createAllForPatientProfile(60, 0, false, setOf(LocationGroup.LIVER_AND_INTRAHEPATIC_BILE_DUCTS)).dropLast(1) +
    PopulationDefinition("55-65y, WHO=0, RAS-") { (diag, epi) ->
        diag.ageAtDiagnosis in 55..65 && epi.whoStatusPreTreatmentStart == 0 && diag.hasRasMutation == false
    }
    
fun pfsForPopulation(population: List<DiagnosisAndEpisode>): String {
    val medianPfs = eventAndCensorshipHistories(population).first.firstOrNull { it.survival <= 0.5 }?.daysSincePlanStart
    return "$medianPfs (n=${population.size})"
}

fun dataFrame(rowLabels: List<Any?>, firstColumnName: String, namedColumns: List<Pair<String, List<String>>>): DataFrame<*> {
    val labelStrings = rowLabels.map { it?.let { it.toString() } ?: "None" }
    return (listOf(firstColumnName to labelStrings) + namedColumns).toMap().toDataFrame()
}

fun pfsTable(patientsByTreatment: Map<String, List<DiagnosisAndEpisode>>, columnDefinitions: List<PopulationDefinition>): DataFrame<*> {
    val allPatients = patientsByTreatment.flatMap { it.value }
    val sortedPatients = patientsByTreatment.entries.sortedByDescending { it.value.size }
                             
    val entries = columnDefinitions.map { (title, criteria) ->
        val populationSize: Int = allPatients.count(criteria)
        val annotatedTitle = "$title (n=$populationSize)"
        annotatedTitle to sortedPatients.map { pfsForPopulation(it.value.filter(criteria)) }
    }
    
    return dataFrame(sortedPatients.map { it.key }, "Treatment", entries)
}

val patientsByTreatment = referencePop.groupBy { (_, episode) -> episode.systemicTreatmentPlan!!.treatment.treatmentGroup.display }
pfsTable(patientsByTreatment, popDefinitions)

In [ ]:
%use kandy

In [ ]:
import com.hartwig.actin.personalization.similarity.population.DiagnosisAndEpisode
import com.hartwig.actin.personalization.similarity.population.PfsCalculation
import org.jetbrains.kotlinx.kandy.ir.Plot

val MIN_PATIENT_COUNT = 20
val percentageArray = (0..100 step 10).toList().map { it / 100.0 to "$it%" }.toTypedArray()

fun createPfsPlot(sortedPopulationsByName: Map<String, List<DiagnosisAndEpisode>>): Plot? {
    val historiesByName = sortedPopulationsByName.mapValues { (_, tumors) -> PfsCalculation.eventHistory(tumors) }
        .filter { (_, histories) -> histories.size >= MIN_PATIENT_COUNT }
        
    historiesByName.mapValues { (_, histories) ->
        val searchIndex = histories.binarySearchBy(-0.5) { -it.survival }
        histories[if (searchIndex < 0) -(searchIndex + 1) else searchIndex]
    }
        .entries.sortedBy { (_, medianEvent) -> medianEvent.daysSincePlanStart!! }
        .forEach { (name, eventCountAndSurvivalAtTime) -> println("$name: ${eventCountAndSurvivalAtTime?.daysSincePlanStart} days") }

    return historiesByName.values.maxOfOrNull { it.last().daysSincePlanStart }?.let { longestInterval ->
        plot {
            step {
                x(historiesByName.flatMap { (_, histories) -> histories.map { it.daysSincePlanStart } })
                {
                    axis.breaksLabeled(*(0..longestInterval step 100).toList().map { it to "$it" }.toTypedArray())
                    axis.name = "Days since treatment start"
                }
                y(
                    historiesByName.flatMap { (_, histories) -> histories.map { it.survival } },
                    "PFS %"
                )
                {
                    axis.breaksLabeled(*percentageArray)
                }
                color(historiesByName.flatMap { (name, histories) -> histories.map { name } }, "Group")
            }
            layout.size = 1000 to 600
        }
    }
}

In [ ]:
createPfsPlot(mapOf(
    "Kaplan-Meier" to referencePop,
    "Ignore censored" to referencePop.filter { (_, episode) -> episode.systemicTreatmentPlan?.hadProgressionEvent == true },
    "Censored as progression" to referencePop.map { (diagnosis, episode) ->
        diagnosis to episode.copy(systemicTreatmentPlan=episode.systemicTreatmentPlan?.copy(hadProgressionEvent=true))
    }
))

In [ ]:
createPfsPlot(popDefinitions.associate { (name, criteria) -> name to referencePop.filter(criteria) })

In [ ]:
createPfsPlot(referencePop.groupBy { (_, episode) -> episode.systemicTreatmentPlan!!.treatment.treatmentGroup.display })

In [ ]:
import com.hartwig.actin.personalization.similarity.population.DiagnosisAndEpisode
import com.hartwig.actin.personalization.datamodel.LocationGroup
import com.hartwig.actin.personalization.datamodel.TreatmentGroup
import com.hartwig.actin.personalization.similarity.population.PopulationDefinition

fun patientHasPfsMetrics(patient: DiagnosisAndEpisode): Boolean {
    val plan = patient.second.systemicTreatmentPlan
    return plan?.observedPfsDays != null && plan.hadProgressionEvent != null
}

fun patientObservedPfsDays(patient: DiagnosisAndEpisode) = patient.second.systemicTreatmentPlan?.observedPfsDays

val populationDefinitions = listOf(
    // PopulationDefinition("hasRenalDisease") { it.first.cciHasRenalDisease == true },
    // PopulationDefinition("!hasRenalDisease") { it.first.cciHasRenalDisease == false },
    // PopulationDefinition("unknown") { it.first.cciHasRenalDisease == null },
    PopulationDefinition("hasMyocardialInfarct") { it.first.cciHasMyocardialInfarct == true },
    PopulationDefinition("!hasMyocardialInfarct") { it.first.cciHasMyocardialInfarct == false },
    PopulationDefinition("unknown") { it.first.cciHasMyocardialInfarct == null },
    // PopulationDefinition("hasMsi") { it.first.hasMsi == true },
    // PopulationDefinition("all") { true },
    // PopulationDefinition("hasBrafMutation") { it.first.hasBrafMutation == true },
    // PopulationDefinition("hasBrafV600EMutation") { it.first.hasBrafV600EMutation == true },
    // PopulationDefinition("hasRasMutation") { it.first.hasRasMutation == true },
    // PopulationDefinition("hasKrasG12CMutation") { it.first.hasKrasG12CMutation == true },
    // PopulationDefinition("!hasMsi") { it.first.hasMsi != true },
    // PopulationDefinition("!hasBrafMutation") { it.first.hasBrafMutation == false },
    // PopulationDefinition("!hasBrafV600EMutation") { it.first.hasBrafV600EMutation == false },
    // PopulationDefinition("!hasRasMutation") { it.first.hasRasMutation == false },
    // PopulationDefinition("!hasKrasG12CMutation") { it.first.hasKrasG12CMutation == false }
)

val filteredPatients = referencePop.filter(::patientHasPfsMetrics)
    // .filter {
    //     it.second.systemicTreatmentPlan!!.treatment.treatmentGroup == TreatmentGroup.PEMBROLIZUMAB
    // }
    .sortedBy(::patientObservedPfsDays)
    
val sortedPatientsByPopulation = populationDefinitions.associate { definition ->
    definition.name to filteredPatients.filter(definition.criteria)
}

// createPfsPlot(sortedPatientsByPopulation)

val groupedPatients = filteredPatients.groupBy { (_, episode) ->
    val metastasesGroups = episode.metastases.map { it.location.locationGroup.display() }.distinct()
    when(metastasesGroups.size) {
        0 -> "no metastases"
        1 -> metastasesGroups.first().let { if (it == "Peritoneal" ) LocationGroup.RETROPERITONEUM_AND_PERITONEUM.display() else it }
        2 -> "2 locations"
        else -> "3+ locations"
    }
}
createPfsPlot(groupedPatients)

In [ ]:
import com.hartwig.actin.personalization.datamodel.TumorType

val metastasesLocations = setOf(LocationGroup.LIVER_AND_INTRAHEPATIC_BILE_DUCTS, LocationGroup.LYMPH_NODES, LocationGroup.BRONCHUS_AND_LUNG)

createPfsPlot(
    patientsByTreatment //.filter { it.key == "FOLFOXIRI-B" || it.key == "FOLFIRI-B" }.toMap()
        .mapValues { (_, value) ->
            value.filter { (diag, epi) ->
                diag.consolidatedTumorType == TumorType.CRC_ADENOCARCINOMA &&
                epi.whoStatusPreTreatmentStart == 0 &&
                epi.metastases.map { it.location.locationGroup }.distinct().count { it in metastasesLocations } >= 2 &&
                // epi.metastases.map { it.location.locationGroup }.distinct().count() >= 2
                diag.ageAtDiagnosis in 35..55
                // diag.hasRasMutation == true
            }
        }
)